# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

## YelpReviewFull 数据集

**Hugging Face 数据集：[ YelpReviewFull ](https://huggingface.co/datasets/yelp_review_full)**

### 数据集摘要

Yelp评论数据集包括来自Yelp的评论。它是从Yelp Dataset Challenge 2015数据中提取的。

### 支持的任务和排行榜
文本分类、情感分类：该数据集主要用于文本分类：给定文本，预测情感。

### 语言
这些评论主要以英语编写。

### 数据集结构

#### 数据实例
一个典型的数据点包括文本和相应的标签。

来自YelpReviewFull测试集的示例如下：

```json
{
    'label': 0,
    'text': 'I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!'
}
```

#### 数据字段

- 'text': 评论文本使用双引号（"）转义，任何内部双引号都通过2个双引号（""）转义。换行符使用反斜杠后跟一个 "n" 字符转义，即 "\n"。
- 'label': 对应于评论的分数（介于1和5之间）。

#### 数据拆分

Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。

## 下载数据集

In [1]:
import os

print(os.environ['HF_HOME'])
print(os.environ['HF_HUB_CACHE'])

E:\BaiduSyncdisk\LearningGPT\hf
E:\BaiduSyncdisk\LearningGPT\hf\hub


In [2]:
from datasets import load_dataset

#dataset = load_dataset(os.path.join(os.environ['HF_HUB_CACHE'], "datasets", "yelp_review_full"))
dataset = load_dataset("yelp_review_full")

d:\ProgramData\anaconda3\envs\transformers\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [4]:
dataset["train"][111]

{'label': 2,
 'text': "As far as Starbucks go, this is a pretty nice one.  The baristas are friendly and while I was here, a lot of regulars must have come in, because they bantered away with almost everyone.  The bathroom was clean and well maintained and the trash wasn't overflowing in the canisters around the store.  The pastries looked fresh, but I didn't partake.  The noise level was also at a nice working level - not too loud, music just barely audible.\\n\\nI do wish there was more seating.  It is nice that this location has a counter at the end of the bar for sole workers, but it doesn't replace more tables.  I'm sure this isn't as much of a problem in the summer when there's the space outside.\\n\\nThere was a treat receipt promo going on, but the barista didn't tell me about it, which I found odd.  Usually when they have promos like that going on, they ask everyone if they want their receipt to come back later in the day to claim whatever the offer is.  Today it was one of th

In [5]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [6]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(dataset["train"])

,label,text
0,2 star,"This place used to be so much fun. Good food. Good service. Not anymore. The local neighborhood feel is gone. No more \""people knowing you name\"". This might as well be another link in a chain restaurant. Food is still pretty good. Service is beyond terrible. So sad to see what was a great place just be so-so."
1,5 stars,"I feel very lucky to have found Camden Tiara as my residence in Las Vegas. The staff are so friendly and remember your name. The grounds are always clean and your work orders for apartment repairs are very timely. The two bedroom units are spacious and after comparing the prices to the other Camden Properties, their rates are a bit lower. Even though you are close to the airport, the sounds are not bad at all. One of the things I have been very grateful for and wanted to give a special shout out to is to Miss Blair, who is an amazing leasing agent. I have a cat who is terrified of being boarded and if I have to travel, it can be an upsetting experience for me and Miss Peaches. Blair will stop by my apartment to check on Peaches and give her can food so that my cat can stay in her comfy home and I can travel with a sound mind. Not very many apartment complexes would go the extra mile for their residents like that. Thank you Camden Tiara for making me feel like I have a home."
2,1 star,"Quantity over quality. What can I say if you want to consume mass quantities for next to nothing come here. Although I did not get sick, the food is marginal at best. You have been warned."
3,5 stars,Guys at Cars NV are awesome heard nothing but great things about the shop. Brought my Mkv Gti in because the check engine light was on and the car was running sluggish. They located the problem and fixed it quickly and for a great price! My Gti runs so good now! Pulls like a freight train again. What really won me over about Cars NV was the amazing customer service & price. Thank you Cars NV crew for the great service and experience see you guys next month for my timing belt/water pump install.
4,3 stars,"well in general it did'nt float my boat i thought the menu especially the happy hour apps were boring. They had a decent selection of beers on tap and the dude behind the bar was very congenial, I did have a burg and fries which I thought were good but for some reason the place leaves me cold like the english weather."
5,2 star,"Was excited to try this place for a late lunch the other day. Owner and staff were very welcoming. Ordered a fish taco, carne asada burrito and fresh raspberry lemonade. While waiting, roamed around the outside ... seating area could be laid out better. Order took only a few minutes and my name was called for pick up. Was suprised at the weight of the taco and burrito ... felt substantial. Tried the fish taco first. It smelled and tasted fishy (not in a good way). Very poorly seasoned fish cooked in dish water. The (flour) taco was loaded with a tasteless mash of beans, lettuce, rice, none of which were very appealing. Looking forward to the carne asada burrito, I was almost equally disappointed. Almost no meat, and what there was of it tasted like old pot roast that had been steamed into a tasteless mush. Again, the burrito was filled with beans, rice and lettuce, but nothing was properly flavored. The tortilla itself tasted old and didn't hold together well, allowing most of the innards to fall out ... you know, when you've let them sit out on the counter for too long. Ended up throwing away 1/2 of the taco and burrito ... something I rarely do. The fresh lemonade tasted like koolaid with slush. Definitely not worth the $3 price. Food prices are cheap, portions large, but quality and taste are not lacking. Two thumbs down! If you want a cheap Mexican fix that tastes good, try La Salsita on Indian School and 32nd Street. Great carne asada and slow cooked pork."
6,5 stars,"My fiance and I came down to Vegas from Portland, OR with the primary intention of getting hitched. Since there 

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [9]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,1 star,"The service was par had to ask for refills and place was not busy!The food was very good, the service was the worst ever! I ordered Ceviche and I never got it, my wife order two chicken taquitos dry nothing on them and Chicken taco no lettuce - cheese only and they messed it up and then the food runner acted like she should take the messed up order. When we got the bill the food server told us she took care of the bill that is correct. When I look through the bill, the server double charged us for the combination plate chicken taco and two taquitos. Very disappointed there is 67 Mexican restrains in a 13 mile ratios. We have a lot of other options next time. They have lots of food servers and food runners, just not a lot of service going on, never got a refill on the salsa and had to ask for water and ice tea refills . We live in green valley, if they are still open in a couple months, might give it a try AGIAN!","[101, 1109, 1555, 1108, 14247, 1125, 1106, 2367, 1111, 1231, 18591, 1116, 1105, 1282, 1108, 1136, 5116, 106, 1109, 2094, 1108, 1304, 1363, 117, 1103, 1555, 1108, 1103, 4997, 1518, 106, 146, 2802, 24664, 11690, 1162, 1105, 146, 1309, 1400, 1122, 117, 1139, 1676, 1546, 1160, 9323, 27629, 21594, 1116, 3712, 1720, 1113, 1172, 1105, 18770, 27629, 2528, 1185, 1519, 7926, 2093, 118, 9553, 1178, 1105, 1152, 20147, 1122, 1146, 1105, 1173, 1103, 2094, 6057, 5376, 1176, 1131, 1431, 1321, 1103, 20147, 1146, 1546, 119, 1332, 1195, 1400, 1103, 4550, 1103, 2094, 9770, 1500, 1366, 1131, 1261, 1920, 1104, 1103, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


### 数据抽样

全量数据训练

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [10]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [11]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [12]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [13]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [15]:
from transformers import TrainingArguments, Trainer
model_dir = "models/bert-base-cased-finetune-yelp"

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  # 实验下来，设置为 20 内存使用还在 16 G以内，且运行效率在 1000 个示例
                                  #   的时候，可以些许提升  train_runtime 193s => 186s
                                  per_device_train_batch_size=20,
                                  num_train_epochs=3,
                                  # 增大 log steps 步数，减少硬盘空间等待
                                  logging_steps=300)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

d:\ProgramData\anaconda3\envs\transformers\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


中途继续的话，可以使用 trainer.train(resume_from_checkpoint=True)

In [17]:
trainer.train()

  0%|          | 300/97500 [04:38<24:52:28,  1.09it/s]

{'loss': 1.1742, 'grad_norm': 6.319441318511963, 'learning_rate': 4.984615384615385e-05, 'epoch': 0.01}


  1%|          | 600/97500 [09:13<24:32:05,  1.10it/s]

{'loss': 0.9862, 'grad_norm': 4.064090728759766, 'learning_rate': 4.969230769230769e-05, 'epoch': 0.02}


  1%|          | 900/97500 [13:47<24:27:34,  1.10it/s]

...

100%|██████████| 97500/97500 [25:32:37<00:00,  1.08it/s]

{'loss': 0.5879, 'grad_norm': 7.096150875091553, 'learning_rate': 0.0, 'epoch': 3.0}


                                                        
100%|██████████| 97500/97500 [25:45:43<00:00,  1.05it/s]

{'eval_loss': 0.7333844304084778, 'eval_accuracy': 0.69282, 'eval_runtime': 785.5531, 'eval_samples_per_second': 63.649, 'eval_steps_per_second': 7.956, 'epoch': 3.0}
{'train_runtime': 92743.9873, 'train_samples_per_second': 21.026, 'train_steps_per_second': 1.051, 'train_loss': 0.6946264202411359, 'epoch': 3.0}


TrainOutput(global_step=97500, training_loss=0.6946264202411359, metrics={'train_runtime': 92743.9873, 'train_samples_per_second': 21.026, 'train_steps_per_second': 1.051, 'train_loss': 0.6946264202411359, 'epoch': 3.0})

In [18]:
test_dataset = tokenized_datasets["test"].shuffle(seed=64)

In [19]:
test_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 50000
})

In [20]:
trainer.evaluate(test_dataset)

100%|██████████| 6250/6250 [12:59<00:00,  8.02it/s]


{'eval_loss': 0.7333844304084778,
 'eval_accuracy': 0.69282,
 'eval_runtime': 780.1249,
 'eval_samples_per_second': 64.092,
 'eval_steps_per_second': 8.012,
 'epoch': 3.0}